#### Library Imports

In [1]:
import re
import time
import numpy as np
import pandas as pd
import os
from datetime import datetime
import logging
from logging.handlers import RotatingFileHandler
from appium import webdriver
from appium.options.android import UiAutomator2Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Configurations